### Задание 1 

In [44]:
from string import punctuation
import numpy as np
from collections import Counter
from nltk.tokenize import sent_tokenize
from gensim.models.phrases import Phrases, Phraser

In [45]:
dostoevsky = open('besy_dostoevsky.txt', encoding='UTF-8').read()

In [46]:
def normalize(text):
    normalized_text = [word.strip(punctuation) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [47]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [48]:
sentences_dostoevsky = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dostoevsky)]

In [49]:
sentences_dostoevsky_small = sentences_dostoevsky[:1000]

In [50]:
unigrams_dostoevsky = Counter()
bigrams_dostoevsky = Counter()
trigrams_dostoevsky = Counter()

for sentence in sentences_dostoevsky_small:
    unigrams_dostoevsky.update(sentence)
    bigrams_dostoevsky.update(ngrammer(sentence))
    trigrams_dostoevsky.update(ngrammer(sentence, n = 3))

In [51]:
id2word_uni_dostoevsky = list(unigrams_dostoevsky)
word2id_uni_dostoevsky = {word:i for i, word in enumerate(id2word_uni_dostoevsky)}

In [52]:
id2word_bi_dostoevsky = list(bigrams_dostoevsky)
word2id_bi_dostoevsky = {word:i for i, word in enumerate(id2word_bi_dostoevsky)}

In [53]:
matrix_dostoevsky_bi = np.zeros((len(unigrams_dostoevsky), 
                   len(unigrams_dostoevsky)))

for ngram in bigrams_dostoevsky:
    word1, word2 = ngram.split()
    matrix_dostoevsky_bi[word2id_uni_dostoevsky[word1]][word2id_uni_dostoevsky[word2]] =  (bigrams_dostoevsky[ngram]/
                                                                     unigrams_dostoevsky[word1])

In [54]:
matrix_dostoevsky_tri = np.zeros((len(bigrams_dostoevsky), 
                   len(unigrams_dostoevsky)))
for ngram in trigrams_dostoevsky:
    word1, word2, word3 = ngram.split()
    bigram = '{} {}'.format(word1, word2)
    matrix_dostoevsky_tri[word2id_bi_dostoevsky[bigram]][word2id_uni_dostoevsky[word3]] =  (trigrams_dostoevsky[ngram]/
                                                                     bigrams_dostoevsky[bigram])

In [55]:
def first_step(text, matrix_2, word2id_bi, id2word, start_idx):
    second_word = np.random.choice(matrix_2.shape[1], p=matrix_2[start_idx])
    word = id2word[second_word]
    text.append(word)
    current_bigram = "<start> {}".format(word)
    current_idx = word2id_bi[current_bigram]
    return current_bigram, current_idx

In [56]:
def generate(matrix, matrix_2, id2word, word2id_bi, word2id_uni, n=100):
    text = []
    start_idx = word2id_uni["<start>"]
    current_bigram, current_idx = first_step(text, matrix_2, word2id_bi, id2word, start_idx)
    for i in range(n):

        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        word = id2word[chosen]
        
        if word == '<end>':
            current_bigram, current_idx = start_step(text, matrix_2, word2id_bi, id2word, start_idx)
        else:
            text.append(word)
            current_bigram = "{} {}".format(current_bigram.split()[1], word)
            current_idx = word2id_bi[current_bigram]
    
    return ' '.join(text)

In [151]:
def generate2(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

#### Триграмная модель

In [154]:
print(generate(matrix_dostoevsky_tri, matrix_dostoevsky_bi, id2word_uni_dostoevsky, word2id_bi_dostoevsky, word2id_uni_dostoevsky, n=100).replace('<end>', '\n'))

бедный иван осипович заговорил отдаленно почти шепотом но всё несколько путался — простонал он машинально не своим голосом сначала степану трофимовичу может быть восторг был преднамеренный а жест нарочно заучен пред зеркалом за полчаса пред чаем но должно быть у него читать газеты и журналы заграничные запрещенные издания и даже в виде новой надежды и когда уже достраивают до самого верху то обладатель положим хоть олимпа убегает в комическом виде а догадавшееся человечество завладев его местом тотчас же по приезде сообщила всё степану трофимовичу являлся постоянно и брал у него раза по два в неделю бывало весело особенно когда сидел летом


#### Биграмная модель

In [152]:
print(generate2(matrix_dostoevsky_bi, id2word_uni_dostoevsky, word2id_uni_dostoevsky).replace('<end>', '\n'))

я — признание замечательное английское семейство les dames charmantes,[20 прелестных дам намеревались прекратить к степану трофимовичу два купец с ним когда великую идею вами давно с вольною федеративною связью об одних лишь бы с эстрады расплакался 
 губернатора человек тем более что главное в удовлетворительном виде если можно так что в самом деле произнесено было написано что он женился в наших глазах как какой-нибудь andrejeff то еще с ним обходилась 
 скажу 
 но это действительно варвара петровна пригласила и был с этим согласен… — очень спокойны и не в день прошло и сообразили 
 они дружески расстались как предсказывал


Вывод:

В триграмной модели предложения получились более связанными и осмысленными. Есть куски, которые очень похожи на человеческую речь. Биграмная модель в сравнении выглядит хуже. Смысла в высказываниях практически нет. 

### Задание 2

In [87]:
text = '''Чикaго — третий по числу жителей (после Нью-Йорка и Лос-Анджелеса) город США, второй по значимости финансовый центр страны (после Нью-Йорка) и крупнейший транспортный узел Северной Америки. 
Расположен на юго-западном побережье озера Мичиган в штате Иллинойс; административный центр округа Кук. Население Чикаго (по данным переписи 2010 года) составляет 2 695 598 человек. Административно город делится на 77-м районов. Агломерация Чикаго (с различными пригородами) называется «Большой Чикаго» или «Страна Чикаго» (англ. Chicagoland; название предложено газетой Chicago Tribune в начале XX века); в ней проживает более 9 млн человек. 
Агломерация Чикаго занимает 37-е место в мире по числу жителей. Чикаго по праву считается экономической, промышленной, транспортной и культурной столицей Среднего Запада. 
Неофициально его иногда также называют «Второй город» и «Город ветров». Впервые Чикаго был назван «Городом ветров» в статье в Chicago Tribune за 1858 год.
30 января 2019 года информационный портал Travel and Leisure опубликовал рейтинг наиболее романтичных городов мира. В ходе исследования анализировалось количество людей, стремящихся сделать предложение руки и сердца либо вступить в брак на территории определённого города. 
Так же анализировалась сексуальная активность жителей и наличие благоприятных условий для ЛГБТ-знакомств. По результатам исследования Чикаго занял пятое место в данном рейтинге.
'''

In [88]:
sentences = sent_tokenize(text)

In [149]:
chars = []
for sent in sentences:
    s = []
    chars.append(s)
    for word in sent:
        for char in word:
            s.append(char)
print(chars[:1])

[['Ч', 'и', 'к', 'a', 'г', 'о', ' ', '—', ' ', 'т', 'р', 'е', 'т', 'и', 'й', ' ', 'п', 'о', ' ', 'ч', 'и', 'с', 'л', 'у', ' ', 'ж', 'и', 'т', 'е', 'л', 'е', 'й', ' ', '(', 'п', 'о', 'с', 'л', 'е', ' ', 'Н', 'ь', 'ю', '-', 'Й', 'о', 'р', 'к', 'а', ' ', 'и', ' ', 'Л', 'о', 'с', '-', 'А', 'н', 'д', 'ж', 'е', 'л', 'е', 'с', 'а', ')', ' ', 'г', 'о', 'р', 'о', 'д', ' ', 'С', 'Ш', 'А', ',', ' ', 'в', 'т', 'о', 'р', 'о', 'й', ' ', 'п', 'о', ' ', 'з', 'н', 'а', 'ч', 'и', 'м', 'о', 'с', 'т', 'и', ' ', 'ф', 'и', 'н', 'а', 'н', 'с', 'о', 'в', 'ы', 'й', ' ', 'ц', 'е', 'н', 'т', 'р', ' ', 'с', 'т', 'р', 'а', 'н', 'ы', ' ', '(', 'п', 'о', 'с', 'л', 'е', ' ', 'Н', 'ь', 'ю', '-', 'Й', 'о', 'р', 'к', 'а', ')', ' ', 'и', ' ', 'к', 'р', 'у', 'п', 'н', 'е', 'й', 'ш', 'и', 'й', ' ', 'т', 'р', 'а', 'н', 'с', 'п', 'о', 'р', 'т', 'н', 'ы', 'й', ' ', 'у', 'з', 'е', 'л', ' ', 'С', 'е', 'в', 'е', 'р', 'н', 'о', 'й', ' ', 'А', 'м', 'е', 'р', 'и', 'к', 'и', '.']]


In [130]:
phrases = Phrases(chars, min_count = 1, scoring='npmi', threshold=1)

In [131]:
p_2 = Phrases(phrases[chars], min_count = 1,scoring='npmi', threshold=-1)

In [135]:
p_3 = Phrases(p_2[phrases[chars]], min_count = 1, scoring='npmi', threshold=-1)

In [137]:
p_4 = Phrases(p_3[p_2[phrases[chars]]], min_count = 1, scoring='npmi', threshold=-1)

In [146]:
p_5 = Phrases(p_4[p_3[p_2[phrases[chars]]]], min_count=1, scoring='npmi', threshold=-1)

In [148]:
print(list(p_5[p_4[p_3[p_2[chars]]]]))

[['Ч_и_к_a_г_о_ _—_ _т_р_е_т_и_й_ ', 'п_о_ _ч_и_с_л_у_ _ж_и_т_е_л_е_й', ' _(_п_о_с_л_е_ _Н_ь_ю_-_Й_о_р_к', 'а_ _и_ _Л_о_с_-_А_н_д_ж_е_л_е_с', 'а_)_ _г_о_р_о_д_ _С_Ш_А_,_ _в_т', 'о_р_о_й_ _п_о_ _з_н_а_ч_и_м_о_с', 'т_и_ _ф_и_н_а_н_с_о_в_ы_й_ _ц_е', 'н_т_р_ _с_т_р_а_н_ы_ _(_п_о_с_л', 'е_ _Н_ь_ю_-_Й_о_р_к_а_)_ _и_ _к', 'р_у_п_н_е_й_ш_и_й_ _т_р_а_н_с_п', 'о_р_т_н_ы_й_ _у_з_е_л_ _С_е_в_е', 'р_н_о_й_ _А_м_е_р_и_к_и_.'], ['Р_а_с_п_о_л_о_ж_е_н_ _н_а_ _ю_г', 'о_-_з_а_п_а_д_н_о_м_ _п_о_б_е_р', 'е_ж_ь_е_ _о_з_е_р_а_ _М_и_ч_и_г', 'а_н_ _в_ _ш_т_а_т_е_ _И_л_л_и_н', 'о_й_с_;_ _а_д_м_и_н_и_с_т_р_а_т', 'и_в_н_ы_й_ _ц_е_н_т_р_ _о_к_р_у', 'г_а_ _К_у_к_.'], ['Н_а_с_е_л_е_н_и_е_ _Ч_и_к_а_г_о', ' _(_п_о_ _д_а_н_н_ы_м_ _п_е_р_е', 'п_и_с_и_ _2_0_1_0_ _г_о_д_а_)_ ', 'с_о_с_т_а_в_л_я_е_т_ _2_ _6_9_5', ' _5_9_8_ _ч_е_л_о_в_е_к_.'], ['А_д_м_и_н_и_с_т_р_а_т_и_в_н_о_ ', 'г_о_р_о_д_ _д_е_л_и_т_с_я_ _н_а', ' _7_7_-_м_ _р_а_й_о_н_о_в_.'], ['А_г_л_о_м_е_р_а_ц_и_я_ _Ч_и_к_а', 'г_о_ _(_с_ _р_а_з_л_и_ч_н_ы_м_и', ' _п_р_и_